# Importing Libraries

In [9]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS



In [12]:
api_key = os.getenv("OPENAI_API_KEY")

In [1]:
llm = OpenAI(openai_api_key = api_key, temperature = 0.9, max_tokens = 500)

# loading the documents

In [12]:
loaders = UnstructuredURLLoader(urls = ["https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
                                        "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load()
len(data)

2

# Splitting The Text

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap = 100
)
docs = text_splitter.split_documents(data)
len(docs)

33

In [18]:
docs[3]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='Reuters\n\nSeptember 12, 2023 / 06:56 IST\n\nWall Street rises as Tesla soars on AI optimism\n\nThe Nasdaq closed sharply higher on Monday as Tesla surged on optimism around artificial intelligence and investors awaited inflation data due later this week.\n\nTeslaÂ (TSLA.O)Â rallied 10% after Morgan StanleyÂ upgradedÂ the electric car maker to "overweight" from "equal-weight," saying its Dojo supercomputer could boost the company\'s market value by nearly $600 billion.')

# Creating OpenAI Embeddings

In [21]:
embeddings = OpenAIEmbeddings()


# Creating Database(FAISS VECTOR INDEX)

In [ ]:
vectorindex_openai = FAISS.from_documents(docs, embeddings)

# Storing VectorIndex into Local

In [ ]:
with open("vector_index.pkl", 'wb') as f:
    pickle.dump(vectorindex_openai , f)

# Loading .pickle file

In [ ]:
with open("vector_index.pkl", "rb") as f:
    vectorIndex = pickle.load(f)

# Creating RetrievalQAWithSourcesChain

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever = vectorIndex.as_retriever())
chain